In [1]:
!pip install geojson

In [2]:
import geojson
import csv

# Load GeoJSON data from a file
with open('gadm41_SAU_0.json', 'r') as geojson_file:
    geojson_data = geojson.load(geojson_file)

# Extract feature properties and write to CSV
csv_filename = 'output1.csv'
with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write CSV header
    properties = geojson_data['features'][0]['properties']
    csv_writer.writerow(properties.keys())

    # Write feature properties to CSV
    for feature in geojson_data['features']:
        properties = feature['properties']
        csv_writer.writerow(properties.values())

print(f'GeoJSON data has been converted to {csv_filename}')


GeoJSON data has been converted to output1.csv


In [4]:
# Step1: to read and print all attributres in json file

import json

# Assuming the JSON file is named 'data.json' and located in the same directory as this script
file_path = 'gadm41_SAU_2.json'

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)

# Function to recursively print all keys and values
def print_json(data, indent=0):
    if isinstance(data, dict):
        for key, value in data.items():
            print('  ' * indent + str(key))
            print_json(value, indent + 1)
    elif isinstance(data, list):
        for item in data:
            print_json(item, indent + 1)
    else:
        print('  ' * indent + str(data))

# Print all data
print_json(data)


type
  FeatureCollection
name
  gadm41_SAU_2
crs
  type
    name
  properties
    name
      urn:ogc:def:crs:OGC:1.3:CRS84
features
    type
      Feature
    properties
      GID_2
        SAU.1.1_1
      GID_0
        SAU
      COUNTRY
        SaudiArabia
      GID_1
        SAU.1_1
      NAME_1
        'Asir
      NL_NAME_1
        عَسِيرٌ
      NAME_2
        Abha
      VARNAME_2
        NA
      NL_NAME_2
        أَبْهَا
      TYPE_2
        Muhafazah
      ENGTYPE_2
        Governorate
      CC_2
        NA
      HASC_2
        NA
    geometry
      type
        MultiPolygon
      coordinates
                41.0265
                17.6643
                41.0262
                17.6654
                41.0278
                17.6659
                41.0283
                17.6643
                41.0265
                17.6643
                41.0251
                17.6839
                41.0243
                17.6835
                41.0237
                17.6847
          

                25.9661
                44.4312
                25.9688
                44.4407
                25.9688
                44.4472
                25.9705
                44.4512
                25.9805
                44.4496
                25.9873
                44.4465
                25.9945
                44.4377
                26.005
                44.4179
                26.0346
                44.4132
                26.0403
                44.4033
                26.0455
                44.3981
                26.0459
                44.3834
                26.0419
                44.3794
                26.0391
                44.3736
                26.0315
                44.3575
                26.0283
                44.3486
                26.0311
                44.3445
                26.0355
                44.3387
                26.0471
                44.2855
                26.0524
                44.2676
                26.0506
                4

                17.8839
                44.1569
                17.8873
                44.1678
                17.8865
                44.2101
                17.8658
                44.2187
                17.8599
                44.2338
                17.8591
                44.2583
                17.8597
                44.2663
                17.8615
                44.2926
                17.8616
                44.3144
                17.8578
                44.312
                17.8309
                44.3063
                17.8221
                44.2911
                17.8119
                44.2709
                17.8043
                44.2645
                17.7974
                44.2538
                17.7797
                44.2525
                17.7714
                44.2418
                17.7392
                44.2234
                17.7044
                44.223
                17.6945
                44.1817
                17.7064
                44

In [5]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [22]:
# Step2: find the lat and long becuse the file just contains geometry


#merge geometry data from one JSON file (gadm41_SAU_2.json) into another (gadm41_SAU_0.json),
#and additionally enrich the destination JSON with latitude and longitude data from the OpenStreetMap API, 

import json
import requests
def get_coordinates_from_osm(name):
    """ Get latitude and longitude from OpenStreetMap by place name. """
    headers = {
        'User-Agent': 'MyResearchProject/1.0 (muhajabh@cardiff.ac.uk)'
    }
    params = {
        'q': name,
        'format': 'json'
    }
    try:
        response = requests.get('https://nominatim.openstreetmap.org/search', params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                return data[0]['lat'], data[0]['lon']
        else:
            print(f"Failed to fetch data: {response.status_code}, {response.text}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
    return None, None

# Example usage
lat, lon = get_coordinates_from_osm('Saudi Arabia')
if lat and lon:
    print(f"Latitude: {lat}, Longitude: {lon}")
else:
    print("No data found or request failed.")

try:
    with open('gadm41_SAU_0.json', 'r') as file:
        source_data = json.load(file)
        geometry = source_data.get('features', [{}])[0].get('geometry', {})

    with open('gadm_0.json', 'r') as file:
        target_data = json.load(file)

    lat, lon = get_coordinates_from_osm('Saudi Arabia')
    if lat and lon:
        target_data[0]['lat'] = lat
        target_data[0]['long'] = lon

    target_data[0]['geometry'] = geometry

    with open('gadm_0_updated.json', 'w') as file:
        json.dump(target_data, file, indent=4)

    print("Updated data saved to 'gadm_0_updated.json'")
except Exception as e:
    print(f"An error occurred: {e}")


Latitude: 25.6242618, Longitude: 42.3528328
Updated data saved to 'gadm_0_updated.json'


In [17]:

import json

def load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            print("Reading content: ", content)  # Debug: print file content to verify
            return json.loads(content)
    except json.JSONDecodeError as e:
        print("JSONDecodeError: ", str(e))
        print("Error at line number:", e.lineno, "Column:", e.colno)
    except Exception as e:
        print("An error occurred:", str(e))

# Try to load your JSON data
data = load_json('gadm_0.json')
if data:
    print("Loaded data:", data)
else:
    print("Failed to load data.")


Reading content:  [
{
    "GID": "SAU",
    "name": "Saudi Arabia",
    "type" : "Country",
    "lat": null,
    "long": null ,
    "within": "",
    "neighbours": [],
    "geometry": {}
    }
]
Loaded data: [{'GID': 'SAU', 'name': 'Saudi Arabia', 'type': 'Country', 'lat': None, 'long': None, 'within': '', 'neighbours': [], 'geometry': {}}]


In [25]:
#to generate the level 1
import json
import requests
import time

def get_coordinates_from_osm(name):
    """ Get latitude and longitude from OpenStreetMap by place name. """
    headers = {
        'User-Agent': 'muhajabh@cardiff.ac.uk'
    }
    params = {
        'q': name,
        'format': 'json'
    }
    try:
        response = requests.get('https://nominatim.openstreetmap.org/search', params=params, headers=headers)
        if response.ok:
            data = response.json()
            if data:
                return data[0]['lat'], data[0]['lon']
            else:
                print(f"No data returned for {name}")
        else:
            print(f"Failed to fetch data for {name}, Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed for {name}: {e}")
    return None, None

def create_custom_json(source_file_path, output_file_path):
    with open(source_file_path, 'r') as file:
        source_data = json.load(file)
    
    new_data = []
    for feature in source_data['features']:
        properties = feature['properties']
        lat, lon = get_coordinates_from_osm(properties.get('NAME_1', ''))
        
        new_item = {
            "GID": properties.get('GID_1', ''),
            "name": properties.get('NAME_1', ''),
            "type": properties.get('ENGTYPE_1', ''),
            "lat": lat,
            "long": lon,
            "within": properties.get('GID_0', ''),
            "neighbours": [],
            "geometry": feature.get('geometry', {})
        }
        new_data.append(new_item)
        time.sleep(1)  # Sleep to prevent hitting rate limits

    with open(output_file_path, 'w') as file:
        json.dump(new_data, file, indent=4)

    print(f"Data successfully saved to {output_file_path}")

# Update the file paths accordingly
create_custom_json('gadm41_SAU_1.json', 'gadm_1_updated.json')


No data returned for AlBahah
No data returned for AlHududashShamaliyah
No data returned for AlJawf
No data returned for ArRiyad
Data successfully saved to gadm_1_updated.json


In [27]:


#to generate the level 1
import json
import requests
import time

def get_coordinates_from_osm(name):
    """ Get latitude and longitude from OpenStreetMap by place name. """
    headers = {
        'User-Agent': 'muhajabh@cardiff.ac.uk'
    }
    params = {
        'q': name,
        'format': 'json'
    }
    try:
        response = requests.get('https://nominatim.openstreetmap.org/search', params=params, headers=headers)
        if response.ok:
            data = response.json()
            if data:
                return data[0]['lat'], data[0]['lon']
            else:
                print(f"No data returned for {name}")
        else:
            print(f"Failed to fetch data for {name}, Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed for {name}: {e}")
    return None, None

def create_custom_json(source_file_path, output_file_path):
    with open(source_file_path, 'r') as file:
        source_data = json.load(file)
    
    new_data = []
    for feature in source_data['features']:
        properties = feature['properties']
        lat, lon = get_coordinates_from_osm(properties.get('NAME_2', ''))
        
        new_item = {
            "GID": properties.get('GID_2', ''),
            "name": properties.get('NAME_2', ''),
            "type": properties.get('ENGTYPE_2', ''),
            "lat": lat,
            "long": lon,
            "within": properties.get('GID_1', ''),
            "neighbours": [],
            "geometry": feature.get('geometry', {})
        }
        new_data.append(new_item)
        time.sleep(1)  # Sleep to prevent hitting rate limits

    with open(output_file_path, 'w') as file:
        json.dump(new_data, file, indent=4)

    print(f"Data successfully saved to {output_file_path}")

# Update the file paths accordingly
create_custom_json('gadm41_SAU_2.json', 'gadm_2_updated.json')




No data returned for AhadRifaydah
No data returned for AlBirk
No data returned for AlMajardah
No data returned for KhamisMushayt
No data returned for RijalAlma
No data returned for SaratUbaida
No data returned for ZahranAl-Janub
No data returned for AlAqiq
No data returned for AlBahah
No data returned for AlHajrah
No data returned for AlMandaq
No data returned for BaniHasan
No data returned for AlUwayqilah
No data returned for Dumaal-Jandal
No data returned for AlHinakiyah
No data returned for WadiAlFara
No data returned for AlBadayea
No data returned for AlMithnab
No data returned for AnNabhaniyah
No data returned for RiyadhAlKhabra
No data returned for UqlatAsSuqur
No data returned for UyunAlJiwa
No data returned for AdDilam
No data returned for AdDiriyah
No data returned for AdDuwadimi
No data returned for AlHariq
No data returned for AlMajmaah
No data returned for AlMuzahimiyah
No data returned for AlQuwayiyah
No data returned for ArRayn
No data returned for AsSulayyil
No data retu

In [71]:
# Step3:to make sure about lat and long, we used the map 

import folium

# Create a map centered on Saudi Arabia
m = folium.Map(location=[23.8859, 45.0792], zoom_start=5)

# Add marker for Al Hudud ash Shamaliyah
folium.Marker([24.50478456116675, 39.69574410811475], popup="Al Hudud ash Shamaliyah").add_to(m)

# Display map
m


In [72]:
# Step4: the prevoise step we prepared the data, now we do the compitinal that required for our model
# so for each level we finds the meast and upodate the file 

import json
import math

def get_atan2(y, x):
    return math.atan2(y, x)

def compute_bearing(endpoint, startpoint):
    x1 = endpoint['lat']
    y1 = endpoint['long']
    x2 = startpoint['lat']
    y2 = startpoint['long']

    radians = get_atan2((y1 - y2), (x1 - x2))

    compass_reading = radians * (180 / math.pi)

    if compass_reading < 0:
        compass_reading = compass_reading + 360

    coord_names = ["N", "E", "S", "W", "N"]
    coord_index = round(compass_reading / 90)

    return coord_names[coord_index]  # returns the coordinate value

def compute_distance(endpoint, startpoint):
    R = 6373.0
    x1 = math.radians(endpoint['lat'])
    y1 = math.radians(endpoint['long'])
    x2 = math.radians(startpoint['lat'])
    y2 = math.radians(startpoint['long'])

    dlon = y2 - y1
    dlat = x2 - x1
    a = (math.sin(dlat / 2)) ** 2 + math.cos(x1) * math.cos(x2) * (math.sin(dlon / 2)) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def find_nearest_neighbors(data):
    for region in data:
        lat = region.get('lat', None)
        lon = region.get('long', None)
        if lat is None or lon is None:
            continue
        startpoint = {'lat': float(lat), 'long': float(lon)}
        
        neighbors = {'N': None, 'E': None, 'S': None, 'W': None}
        
        for other_region in data:
            if other_region == region:
                continue
            other_lat = other_region.get('lat', None)
            other_lon = other_region.get('long', None)
            if other_lat is None or other_lon is None:
                continue
            endpoint = {'lat': float(other_lat), 'long': float(other_lon)}
            distance = compute_distance(endpoint, startpoint)
            direction = compute_bearing(endpoint, startpoint)
            
            if neighbors[direction] is None or distance < neighbors[direction]['distance']:
                neighbors[direction] = {'name': other_region['name'], 'distance': distance}
        
        region['neighbours'] = neighbors

# Load the JSON data
#with open('gadm_1_updated.json', 'r') as file:
with open('gadm_2_updated_latlong.json', 'r') as file:
    data = json.load(file)

# Find the nearest neighbors for each region
find_nearest_neighbors(data)

# Save the updated data to a new JSON file
# with open('gadm_1_updated_nearest.json', 'w') as file:
with open('gadm_2_updated_nearest.json', 'w') as file:
    json.dump(data, file, indent=4)

print("Updated data saved to 'gadm_1_updated_nearest.json'")


Updated data saved to 'gadm_1_updated_nearest.json'


In [60]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [67]:
# find lat and long for each polygon in level 2<<< centroied of each polygon 

import json
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

# Load JSON data from file
with open('gadm_2_updated.json', 'r') as file:  
    data = json.load(file)

# Function to calculate centroids and update the data
def update_centroids(data):
    for item in data:
        coordinates = item['geometry']['coordinates']
        
        # Create a list of polygons from the coordinates
        polygons = []
        for multi_poly in coordinates:
            for poly in multi_poly:
                polygons.append(Polygon(poly))
        
        # Create a MultiPolygon
        multi_polygon = MultiPolygon(polygons)
        
        # Calculate the centroid
        centroid = multi_polygon.centroid
        
        # Update lat and long in the data
        item['lat'] = centroid.y
        item['long'] = centroid.x
    
    return data

# Update centroids in the data
updated_data = update_centroids(data)

# Save the updated data back to a JSON file
with open('gadm_2_updated_latlong.json', 'w') as file:
    json.dump(updated_data, file, indent=4)

# Print updated data
print(json.dumps(updated_data, indent=4))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

